### Парсер сайтa filmdoo.com 

Задача была спарсить инфо по фильмам производства Тайланда. В этот раз нужно было использовать Selenium, т.к.
сайт рендерится через Jacascript. Также использовал таймауты, чтобы не попасть под бан сервера

In [1]:
#начинаем с импорта библиотек
from selenium import webdriver
from  bs4 import BeautifulSoup as bs
from urllib.request import urljoin
from tqdm.notebook import tqdm
import pandas as pd
import time
import re
import csv

In [84]:
# Задаем глобальные параметры
domen = 'https://www.filmdoo.com/'
url='https://www.filmdoo.com/thai/page/1'
filename = 'filmdoo-links.csv'
driver = webdriver.Chrome(executable_path ='c:../Webdriver/bin/chromedriver.exe')

In [15]:
# Проверяем готовность супа через селениум (1)
#url='https://www.filmdoo.com/thai/page/6'
driver.get(url)
time.sleep(10) #пауза 10 секунд
html = driver.page_source
soup = bs(html,'lxml')
#print(soup)

In [31]:
# Генератор пагинации (2)
for i in range(1,17):
    url = f'https://www.filmdoo.com/thai/page/{i}'

https://www.filmdoo.com/thai/page/1
https://www.filmdoo.com/thai/page/2
https://www.filmdoo.com/thai/page/3
https://www.filmdoo.com/thai/page/4
https://www.filmdoo.com/thai/page/5
https://www.filmdoo.com/thai/page/6
https://www.filmdoo.com/thai/page/7
https://www.filmdoo.com/thai/page/8
https://www.filmdoo.com/thai/page/9
https://www.filmdoo.com/thai/page/10
https://www.filmdoo.com/thai/page/11
https://www.filmdoo.com/thai/page/12
https://www.filmdoo.com/thai/page/13
https://www.filmdoo.com/thai/page/14
https://www.filmdoo.com/thai/page/15
https://www.filmdoo.com/thai/page/16


In [16]:
#Блок эктракции ссылок с одного грида (3)

links=[]
sec = soup.find('div', class_='films-section')
l = sec.find_all('div', class_=re.compile('sort-film '))
for i in l:
    ll = i.find('a')['href']
    links.append(ll)
print(links)

['https://www.filmdoo.com/films/rurouni-kenshin/', 'https://www.filmdoo.com/films/the-victim/', 'https://www.filmdoo.com/films/until-now/', 'https://www.filmdoo.com/films/the-unseeable/', 'https://www.filmdoo.com/films/two-brothers/', 'https://www.filmdoo.com/films/the-tiger-blade/', 'https://www.filmdoo.com/films/video-clip/', 'https://www.filmdoo.com/films/wonderful-town/', 'https://www.filmdoo.com/films/the-giant-king/', 'https://www.filmdoo.com/films/hello-yasothorn/', 'https://www.filmdoo.com/films/thai-thief/', 'https://www.filmdoo.com/films/the-tesseract/', 'https://www.filmdoo.com/films/tears-of-the-black-tiger/', 'https://www.filmdoo.com/films/red-eagle/', 'https://www.filmdoo.com/films/rahtree-reborn/', 'https://www.filmdoo.com/films/queens-of-langkasuka/', 'https://www.filmdoo.com/films/ploy/', 'https://www.filmdoo.com/films/pisaj/', 'https://www.filmdoo.com/films/p-047/', 'https://www.filmdoo.com/films/p/', 'https://www.filmdoo.com/films/at-the-gate-of-the-ghost/', 'https:/

In [ ]:
# Сборка пагинатора (3 в 1):
links=[]
for i in tqdm(range(1,17), desc='Parsing grids', leave=False):
    url = f'https://www.filmdoo.com/thai/page/{i}'
    driver.get(url)
    time.sleep(10) # будем перегружать грид раз в 10 секунд
    html = driver.page_source
    soup = bs(html,'lxml')
    sec = soup.find('div', class_='films-section')
    l = sec.find_all('div', class_=re.compile('sort-film '))
    for i in l:
        ll = i.find('a')['href']
        links.append(ll)
    #links.append(url) # для проверки выгрузки   
save_products(links)
print('DONE!')

In [3]:
def save_products (result):
    with open(filename, 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.writer(outfile, lineterminator='\n')
        spamwriter.writerow(['Product_Link']) 
        for i in result:
            spamwriter.writerow([i])

# Функция загрузки файла со ссылками на целевые страницы
def open_csv(filename):
    results=[]
    with open(filename, 'r') as data:
        data.readline().rstrip()
        for line in data:
            results.append(line[:-1])
    return results

def save_product_info(products):
    with open('thai-films.csv', 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.DictWriter(outfile, fieldnames=get_field_name(products), lineterminator='\n')    
        spamwriter.writeheader() #записывет заголовки столбцов равные ключам словаря
        spamwriter.writerows(products)
        
# Вспом.функция для сбора названия полей с словарях с продуктами для формирования заголовка в CSV (Без сортировки)
def get_field_name(products):
    field_names = set()
    for i in products:
        for k in i:
            field_names.add(k)
    return field_names


In [236]:
# Посмотрим, как все будет выглядеть словарь с инфой по фильму
params

{'1_title': 'Pee Mak Phrakanong',
 '2_year': '2013',
 '3_length': '115 minutes',
 '4_desc': "Mak served in the war during the beginning of the Rattanakosin Dynasty. At war he became friends with Ter, Puak, Shin, and Aey, whose lives he saved. Once the war was over, Mak invited his four friends into his home in Phra Khanong town and introduced them to his beautiful wife Nak and his newborn baby boy Dang. A rumor was going around in the village that Nak had died giving birth to her stillborn baby, Dang. The source of this rumour; Aunty Priak, owner of local liquor store was found dead floating on the river a few days later. The four friends felt it's time to tell Mak the possibility of Nak and Dang could be dead and lingered as haunting ghosts and risk their lives. It's up to Mak to choose love or reality.",
 '5_rating': '3.5',
 '6_url': 'https://www.filmdoo.com/films/pee-mak-phrakanong/',
 'Region': ['South East Asia'],
 'Country': ['Thailand'],
 'Language': ['Thai'],
 'Genre': ['Comedy

In [85]:
# Основной блок парсера
results = open_csv(filename)  
films_info=[]
for url in tqdm(results, desc='Getting info from film page', leave=False):  
    driver.get(url)
    time.sleep(5)
    html = driver.page_source
    soup = bs(html,'lxml')
    params={}
    try:
        a=soup.find('h1', class_='film-name').text
        if a:
            params['1_title']=a
        else:
            params['1_title']=a
        b=soup.find('div', class_='newtheme-countery').text
        if b:
            params['2_year']=b.split(' ')[1]
            params['3_length']=b.split(' ')[2]+" "+'minutes'
        params['6_url']=url
        d=soup.select('#toper > div > div.newtheme-film-profile > div.newtheme-container > div.newtheme-co-left > div.newtheme-rate > form > div > div.caption > span')
        for i in d:
            dd = i.text       
        if dd:
            params['5_rating']=dd     
        c=soup.find('span', itemprop='description').text
        if c:
            params['4_desc']=c
    except:
        print('Часть тегов не обнаружена. Двигаюсь дальше')
        pass
    try:
        b=soup.find('div', class_='newtheme-co-left').find_all('div', class_='newtheme-filmteam')
        lst=[]
        left_block={}
        for i in b:
            y = i.find('div', class_='filmteam-title').text.split(' ')[0]
            w = i.find('a', class_='filmteam-para').text
            cnt = len(i)
            s={y:w}
            lst.append(s)
            left_block[y]=[w]
            if cnt>2:
                tag_a = i.find_all('a')
                for v in tag_a:
                    ss={y:v.text}
                    lst.append(ss)
                    if y in params:
                        left_block[y].append(v.text)
    except:
        pass
    params.update(left_block)
    films_info.append(params)
save_product_info(films_info)

Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов не обнаружена. Двигаюсь дальше
Часть тегов

Из-за сбоя, парсил двумя частями

In [81]:
# Импорт первой части
df = pd.DataFrame(films_info)
df.head(2)

,1_title,2_year,3_length,4_desc,5_rating,6_url,Region,Country,Language,Subtitles,...,Writer,Tags,DOP,Distributor,Production,Quality,Festivals,Source:,Awards,Locations
0,From Bangkok to Mandalay,Thailand,2016 minutes,Ten letters are sent from a man in Myanmar to ...,5,https://www.filmdoo.com/films/from-bangkok-to-...,[South East Asia],[Myanmar],[Thai],[English],...,[Chartchai Ketnust],[love],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Last Executioner,2014,95 minutes,"Inspired by real events, The Last Executioner ...",4.5,https://www.filmdoo.com/films/the-last-executi...,[South East Asia],[Thailand],[Thai],[English],...,[Don Linder],[based on true story],[Wade Muller],[De Warrenne Pictures],[De Warrenne Pictures],[HD],NaN,NaN,NaN,NaN


In [90]:
# Импорт второй части
df2 = pd.DataFrame(films_info)
df2.head(2)

,1_title,2_year,3_length,6_url,5_rating,Country,Language,Genre,Cast,Director,...,Producer,4_desc,DOP,Source:,Awards,Distributor,Festivals,Subtitles,Locations,Quality
0,Love Beat,2016,111 minutes,https://www.filmdoo.com/films/love-beat/,3,[Thailand],[Thai],[Music],[Benjaphol Choey-arun],[Prachya Pinkaew],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Unborn Child,2011,NaN,NaN,NaN,[Thailand],[Thai],[Horror],[Somchai Kemglad],[Poj Arnon],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
df.shape

(180, 24)

In [95]:
df2.shape

(312, 24)

In [96]:
# объеденяем первую и вторую части
final_df = pd.concat([df, df2], ignore_index=True, sort=False)

In [97]:
final_df.shape

(492, 24)

In [100]:
final_df.sample(25)

,1_title,2_year,3_length,4_desc,5_rating,6_url,Region,Country,Language,Subtitles,...,Writer,Tags,DOP,Distributor,Production,Quality,Festivals,Source:,Awards,Locations
463,Pee Mak Phrakanong,2013,115 minutes,Mak served in the war during the beginning of ...,3.5,https://www.filmdoo.com/films/pee-mak-phrakanong/,[South East Asia],[Thailand],[Thai],NaN,...,[Chantavit Dhanasevi],[ghost],NaN,NaN,NaN,NaN,NaN,[Official site],NaN,NaN
56,Insidious: Chapter 3,USA,2015 minutes,A twisted new tale of terror begins for a teen...,3,https://www.filmdoo.com/films/insidious-chapte...,NaN,[Canada],[English],NaN,...,[Leigh Whannell],NaN,[Brian Pearson],NaN,[Sony Pictures],NaN,NaN,NaN,NaN,NaN
34,Uncle Boonmee Who Can Recall His Past Lives,2010,114 minutes,The latest film from the director of Syndromes...,4.5,https://www.filmdoo.com/films/uncle-boonmee-wh...,[South East Asia],[Thailand],[Thai],[English],...,[Apichatpong Weerasethakul],NaN,NaN,[New Wave Films],NaN,[HD],[Cannes Official Selection],NaN,[Cannes Palme d'Or],NaN
287,999-9999,2002,103 minutes,Rainbow has just moved to an international sch...,3.5,https://www.filmdoo.com/films/999-9999/,[South East Asia],[Thailand],[Thai],NaN,...,[Peter Manus],[death],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,Rurouni Kenshin,2012,134 minutes,Former legendary assassin Kenshin Himura has n...,5,https://www.filmdoo.com/films/rurouni-kenshin/,[North East Asia],[Japan],[Japanese],NaN,...,NaN,[martial arts],NaN,NaN,[Warner Bros.],NaN,NaN,NaN,NaN,NaN
279,Last Life in the Universe,Thailand,2003 minutes,An obsessive-compulsive Japanese librarian liv...,5,https://www.filmdoo.com/films/last-life-in-the...,[South East Asia],[Japan],[English],NaN,...,NaN,[brother brother relationship],[Christopher Doyle],NaN,[Cinemasia],NaN,NaN,NaN,NaN,NaN
256,Dark Heaven,1958,102 minutes,"A poor orphan girl, Nien, is on the run after ...",0,https://www.filmdoo.com/films/dark-heaven/,[South East Asia],[Thailand],[Thai],NaN,...,[Ratana Pestonji],NaN,NaN,NaN,[Kachara Film],NaN,NaN,NaN,NaN,NaN
2,Sway,USA,France minutes,"Three cities, three couples, three intriguing ...",4,https://www.filmdoo.com/films/sway/,[South East Asia],[Thailand],[English],[English],...,[Rooth Tang],[triptych],[Lyn Moncrief],NaN,NaN,NaN,[Toronto],NaN,NaN,NaN
457,Seven Something,2012,147 minutes,Seven Something is a love story and was shot b...,3.5,https://www.filmdoo.com/films/seven-something/,[South East Asia],[Thailand],[Thai],NaN,...,[Jira Maligool],NaN,[Naruphol Chokanapitak],NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,Deep in the Jungle,2008,91 minutes,"Nawin, an army secret service agent decided to...",0,https://www.filmdoo.com/films/deep-in-the-jungle/,NaN,[Thailand],[Thai],NaN,...,[Teerawat Rujintham],NaN,NaN,NaN,[Pranakorn Films],NaN,NaN,NaN,NaN,NaN


In [101]:
final_df.columns

Index(['1_title', '2_year', '3_length', '4_desc', '5_rating', '6_url',
       'Region', 'Country', 'Language', 'Subtitles', 'Genre', 'Cast',
       'Director', 'Producer', 'Writer', 'Tags', 'DOP', 'Distributor',
       'Production', 'Quality', 'Festivals', 'Source:', 'Awards', 'Locations'],
      dtype='object')

In [99]:
# Эксопрт полных результатов в csv
final_df.to_csv('thai_films_full.csv')